In [4]:
from urllib.request import urlretrieve
import zipfile
import pandas as pd
import numpy as np

users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    'data/u.user', sep='|', names=users_cols, encoding='latin-1')

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'data/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

# The movies file contains a binary feature for each genre.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western",
]
movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    'data/u.item', sep='|', names=movies_cols, encoding='latin-1')

# Since the ids start at 1, we shift them to start at 0.
users["user_id"] = users["user_id"].apply(lambda x: str(x-1))
movies["movie_id"] = movies["movie_id"].apply(lambda x: str(x-1))
movies["year"] = movies['release_date'].apply(lambda x: str(x).split('-')[-1])
ratings["movie_id"] = ratings["movie_id"].apply(lambda x: str(x-1))
ratings["user_id"] = ratings["user_id"].apply(lambda x: str(x-1))
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))


genre_occurences = movies[genre_cols].sum().to_dict()

genres_encoded = {x: i for i, x in enumerate(genre_cols)}



def get_genres(movies, genres):
    def get_all_genres(gs):
        active = [str(genres_encoded[genre]) for genre, g in zip(genres, gs) if g==1]
        if len(active) == 0:
            return '0'
        return ','.join((active))
    movies['all_genres'] = [
        get_all_genres(gs) for gs in zip(*[movies[genre] for genre in genres])]

get_genres(movies, genre_cols) # 각 유저가 본 장르 얻기

rating_details_sample = ratings.merge(movies, on='movie_id').merge(users, on='user_id')

rating_details_sample['user_id']=rating_details_sample['user_id'].astype(int)
rating_details_sample['movie_id']=rating_details_sample['movie_id'].astype(int)
rating_details_sample=rating_details_sample.set_index(['user_id','unix_timestamp']).sort_index()
rating_details_sample =rating_details_sample.reset_index()

rating_details_sample['movie_type']=np.where(rating_details_sample['rating'] >= 3, 'like', 'dislike') # 3보다 크면 like
rating_details_sample['movie_name']=rating_details_sample['title'].str[:-6] # 년도 부분 자르기

user_ids = rating_details_sample["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

movie_ids = rating_details_sample["movie_id"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}


title_ids = rating_details_sample["movie_name"].unique().tolist()
title2title_encoded = {x: i for i, x in enumerate(title_ids)}
title_encoded2title = {i: x for i, x in enumerate(title_ids)}

rating_details_sample["user"] = rating_details_sample["user_id"].map(user2user_encoded)
rating_details_sample["movie"] = rating_details_sample["movie_id"].map(movie2movie_encoded)
rating_details_sample["title_d"] = rating_details_sample["movie_name"].map(title2title_encoded)

sample_data=rating_details_sample[['user','occupation','sex']]
sample_data=sample_data.reset_index()

movie_list = rating_details_sample.groupby(['user','movie_type'])['movie'].apply(list).reset_index()
title_list = rating_details_sample.groupby(['user'])['title_d'].apply(list).reset_index()
genre_list = rating_details_sample.groupby(['user'])['all_genres'].unique().apply(list).reset_index()

# Get the unique set of genre for all the users
genre_list['all_genres']=genre_list['all_genres'].apply(lambda x: list(set(','.join(x))) ) # 중복제거
genre_list['all_genres']=genre_list['all_genres'].apply(lambda x:[ x for x in x if x.isdigit() ])

user_video_list = movie_list.pivot(index='user', columns='movie_type', values='movie').reset_index()
user_video_list.fillna(rating_details_sample["movie"].max()+1, inplace=True)

sample_data = sample_data.drop('index',axis=1)
sample_data = sample_data.drop_duplicates()

user_final_list = pd.merge(user_video_list,title_list, how= 'left')
user_title_list1 = pd.merge(user_final_list,genre_list, how='left')
user_title_list = pd.merge(user_title_list1,sample_data, how='left')

user_title_list['like'] =user_title_list['like'].apply(lambda x: x if type(x) is list else [x])
user_title_list['dislike'] =user_title_list['dislike'].apply(lambda x: x if type(x) is list else [x])
user_title_list['predict_labels'] = user_title_list['like'].apply(lambda x: (x[-1])) #label을 마지막 값으로..
user_title_list['like']=user_title_list['like'].apply(lambda x: (x[:-1]))

user_title_list['predict_labels'] = user_title_list['predict_labels'].apply(lambda x: 1 if x == 10 else x)
print(user_title_list['predict_labels'])
user_title_list_e=user_title_list[(user_title_list.user >= 1)&
                                  (user_title_list.user <= 500)]

0    3
1    5
2    6
3    1
4    1
5    8
6    6
7    4
8    1
9    1
Name: predict_labels, dtype: int64


In [5]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

In [6]:
user_title_list_e['title_d']

1       [4, 5]
2       [0, 6]
3          [6]
4          [7]
5          [8]
6       [5, 6]
7    [2, 9, 4]
8          [9]
9          [9]
Name: title_d, dtype: object

In [7]:
user_title_list_e['predict_labels']

1    5
2    6
3    1
4    1
5    8
6    6
7    4
8    1
9    1
Name: predict_labels, dtype: int64

In [29]:
num_class = rating_details_sample["movie"].max() + 2
embedding_dims = 16
model = keras.Sequential()
model.add(keras.layers.Embedding(input_dim=num_class, output_dim=embedding_dims, input_shape=(None,)))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 16)          176       
_________________________________________________________________
global_average_pooling1d_5 ( (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 465
Trainable params: 465
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
 model.fit(tf.keras.preprocessing.sequence.pad_sequences(user_title_list_e['title_d']),
                    user_title_list_e['predict_labels'].values,
                    epochs=40)

Epoch 1/40
1/1 [==============================] - 0s 1ms/step - loss: 0.6784 - accuracy: 0.4444
Epoch 2/40
1/1 [==============================] - 0s 1ms/step - loss: 0.6551 - accuracy: 0.4444
Epoch 3/40
1/1 [==============================] - 0s 1ms/step - loss: 0.6317 - accuracy: 0.4444
Epoch 4/40
1/1 [==============================] - 0s 943us/step - loss: 0.6076 - accuracy: 0.4444
Epoch 5/40
1/1 [==============================] - 0s 810us/step - loss: 0.5835 - accuracy: 0.4444
Epoch 6/40
1/1 [==============================] - 0s 1ms/step - loss: 0.5599 - accuracy: 0.4444
Epoch 7/40
1/1 [==============================] - 0s 2ms/step - loss: 0.5359 - accuracy: 0.4444
Epoch 8/40
1/1 [==============================] - 0s 1ms/step - loss: 0.5129 - accuracy: 0.4444
Epoch 9/40
1/1 [==============================] - 0s 1ms/step - loss: 0.4896 - accuracy: 0.4444
Epoch 10/40
1/1 [==============================] - 0s 1ms/step - loss: 0.4662 - accuracy: 0.4444
Epoch 11/40
1/1 [==================

In [12]:
rating_details_sample["movie"].max() + 2

11